In [1]:
import models as m

# Welcome! In this tutorial we will make a hybrid DNA cage ligated to KDPG Aldolase

## We will use the KDPG files generated in the KDPG Example Notebook as the protein

## Also we have a pre-relaxed DNA cage structure we provide

## Here is our pre-relaxed DNA cage

![dna cage](./examples/imgs/dcage.png)


## And our KDPG Aldolase
    
    In pink are the Cysteine residues used to ligate the DNA to the protein

![kdpg ligate](./examples/imgs/kdpgligate.png)

## NOTE: Mutual traps between the ends of each arm were added during the relaxtion to get the cage into approximately the right shape


# Combining the Structures

    !!!! MUST BE DONE IN THE FOLLOWING ORDER TO GET CORRECT SIMULATION FILES !!!!
    
    Drag and drop 'dnacage' .top and .dat files into oxView
    Drag and drop 'kdpg' .top and .dat files into oxView
    
    We have:
![cage assembly](./examples/imgs/cageassemble.png)


## Now we select the protein system in oxView, translate and rotate it to have the modified residues as close to the DNA arms as we can get (doesn't have to be perfect)

SELECTED:

![cage mov](./examples/imgs/cageselect.png)


ROTATED and TRANSLATED:

![rot and tran](./examples/imgs/cageplaced.png)

## Drop the kdpg.par file in and download the simulation files

## Reopen the simulation files in a fresh instance of oxView




## To finish setting up the linker between the DNA and protein we sequentially select each modified protein residue then the DNA base it is linked to


![cage trap](./examples/imgs/cagetrap.png)


## Simply Click 'Selected Base Trap File' to download the mutual trap file

    This is our mutual trap file
    
![trap file](./examples/imgs/trapfile.png)


## We now edit the r0 (equilbrium distance) and stiff (force constant) parameters to match fully atomistic simulations of our linker (LC-SPDP)

    (in practice you would also take into account length of the amine modification on the DNA
    and the cysteine side length, however that is ignored here)

    Edited Trap file
    
![etrap file](./examples/imgs/etrapfile.png)

## Going back to our system in oxView, we must do the following to generate a correct topology, dat, and parameter file



